In [1]:
import numpy as np
import pandas as pd
import torch
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import sys, os


os.chdir("/Users/samuel.gruffaz/Documents/PEcollab")

from leaspy.api import Leaspy

from leaspy import Leaspy, Data, AlgorithmSettings, Plotter, Dataset, IndividualParameters
from leaspy.models.utils.attributes.attributes_factory import AttributesFactory
from leaspy.algo.algo_factory import AlgoFactory
from leaspy.utils.visualisation.visual_update_b import *

max benton 30
HVLT 60 MOCA 30 SCOPA-AUT 73 ADL 100


In [22]:
os.chdir("/Users/samuel.gruffaz/Documents/data")
df = pd.read_csv("TAD_ADAS_VEN_HIPP_ENTHORINAL.csv", dtype={'ID':str})
df.set_index(['ID','TIME'], inplace=True)
os.chdir("/Users/samuel.gruffaz/Leaspype_kernel")

print(df.shape)

(4262, 4)


In [2]:
os.chdir("/Users/samuel.gruffaz/Documents/data")
dtadpole=pd.read_csv("TADPOLE_D1_bio_TD1_norm.csv", sep=";")
dtadpole.rename(columns={"RID":"ID", "AGE_vis":"TIME"}, inplace=True)
dtadpole.set_index(["ID", "TIME"], inplace=True)
print(dtadpole.head(5))

            Phase VISCODE2    EXAMDATE  YEARS_SINCE_bl  DX_seq  DX_ffill  \
ID TIME                                                                    
2  74.300   ADNI1       bl  2005-09-08        0.000000      10       0.0   
   74.790   ADNI1      m06  2006-03-06        0.490075      10       0.0   
   77.268   ADNI1      m36  2008-08-27        2.967830      10       0.0   
   79.338  ADNIGO      m60  2010-09-22        5.037650      10       0.0   
   80.329   ADNI2      m72  2011-09-19        6.028750      10       0.0   

           SITEID   DX  DX_change DX_full_bl  ...  MES_MRI_MIDTEMPORAL_ICV  \
ID TIME                                       ...                            
2  74.300      11  0.0        1.0         CN  ...                 0.355949   
   74.790      11  0.0        1.0         CN  ...                      NaN   
   77.268      11  0.0        1.0         CN  ...                      NaN   
   79.338      11  0.0        1.0         CN  ...                      NaN   

In [ ]:
TADPOLE_FTS = {'ADAS_TOT13': 'ADAS13', 
               'MMSE_TOT': 'MMSE', 
               'MES_MRI_VENTRICLES_ICV': 'Ventricles_ICV',
               'MES_MRI_HIPPOCAMPUS_ICV': 'Hippocampi_ICV',
               'MES_PET_FDG_AVG': 'FDG',
              }

In [79]:
dtadpole["DX_seq"].astype(str).str.contains("2")
alz_index = dtadpole["DX_seq"]==2

tad=dtadpole[alz_index]
tad1=tad[["MMSE_TOT",  "MES_MRI_HIPPOCAMPUS_ICV", "MES_MRI_VENTRICLES_ICV"
            ] ]
print(tad1.shape)
tad2=tad1.dropna()
print(tad2.shape)

(1217, 3)
(732, 3)


In [61]:

os.chdir("/Users/samuel.gruffaz/Documents/data")

dHVLT=pd.read_csv("HVLT.csv").set_index(['ID', 'TIME'])
dMOCA=pd.read_csv("MOCA.csv").set_index(['ID', 'TIME'])
dSCOPAAUT=pd.read_csv("SCOPA-AUT.csv").set_index(['ID', 'TIME'])
dBenton=pd.read_csv("Benton_JLO.csv").set_index(['ID', 'TIME'])
dADL=pd.read_csv("Schwab_and_England_ADL.csv").set_index(['ID', 'TIME'])

In [62]:
print(dHVLT.columns)
print(dMOCA.columns)
print(dSCOPAAUT.columns)
print(dBenton.columns)
print(dADL.columns)
dADL['SCHWAB_AND_ENGLAND']=1-dADL['SCHWAB_AND_ENGLAND']/100.
dBenton['BENTON_JLO']=1-dBenton['BENTON_JLO']/30.
dMOCA['MOCA_TOTAL']=1-dMOCA['MOCA_TOTAL']/30.
dHVLT['HVLT_TOTAL']=1-dHVLT['HVLT_TOTAL']/60.
dSCOPAAUT['SCOPA_TOTAL']=dSCOPAAUT['SCOPA_TOTAL']/73.0

Index(['HVLT_TOTAL', 'HVLT_1', 'HVLT_2', 'HVLT_3', 'HVLT_4', 'HVLT_5'], dtype='object')
Index(['MOCA_TOTAL'], dtype='object')
Index(['SCOPA_TOTAL', 'SCOPA_1', 'SCOPA_2', 'SCOPA_3', 'SCOPA_4', 'SCOPA_5',
       'SCOPA_6', 'SCOPA_7', 'SCOPA_8', 'SCOPA_9', 'SCOPA_10', 'SCOPA_11',
       'SCOPA_12', 'SCOPA_13', 'SCOPA_14', 'SCOPA_15', 'SCOPA_16', 'SCOPA_17',
       'SCOPA_18', 'SCOPA_19', 'SCOPA_20', 'SCOPA_21', 'SCOPA_22', 'SCOPA_23',
       'SCOPA_24', 'SCOPA_25', 'SCOPA_26'],
      dtype='object')
Index(['BENTON_JLO'], dtype='object')
Index(['SCHWAB_AND_ENGLAND'], dtype='object')


In [63]:
dHVLTtot=dHVLT.drop(['HVLT_1', 'HVLT_2', 'HVLT_3', 'HVLT_4',
       'HVLT_5'],axis=1)
dSCOPAAUTtot=dSCOPAAUT.drop(['SCOPA_1', 'SCOPA_2', 'SCOPA_3', 'SCOPA_4',
       'SCOPA_5', 'SCOPA_6', 'SCOPA_7', 'SCOPA_8', 'SCOPA_9', 'SCOPA_10',
       'SCOPA_11', 'SCOPA_12', 'SCOPA_13', 'SCOPA_14', 'SCOPA_15', 'SCOPA_16',
       'SCOPA_17', 'SCOPA_18', 'SCOPA_19', 'SCOPA_20', 'SCOPA_21', 'SCOPA_22',
       'SCOPA_23', 'SCOPA_24', 'SCOPA_25', 'SCOPA_26'],axis=1)


In [64]:
print(dSCOPAAUTtot.head(5))
print(dBenton.head(5))
print(dHVLT.head(10))
print(dMOCA.head(30))
print(dADL.head(30))

                SCOPA_TOTAL
ID   TIME                  
3000 70.123631     0.082192
     71.123631     0.123288
     72.123631     0.123288
     73.123631     0.109589
     74.123631     0.164384
                BENTON_JLO
ID   TIME                 
3000 70.123631    0.000000
     71.123631    0.000000
     72.123631    0.066667
     73.123631    0.133333
     74.123631    0.066667
                HVLT_TOTAL  HVLT_1  HVLT_2  HVLT_3  HVLT_4  HVLT_5
ID   TIME                                                         
3000 70.123631    0.133333     8.0    11.0    11.0    10.0    12.0
     71.123631    0.216667     7.0     9.0    10.0    10.0    11.0
     72.123631    0.166667     6.0    11.0    11.0    10.0    12.0
     73.123631    0.150000     7.0    10.0    11.0    11.0    12.0
     74.123631    0.133333     8.0    12.0    10.0    10.0    12.0
     75.123631    0.166667     9.0    10.0    10.0    10.0    11.0
     77.123631         NaN     5.0    10.0    11.0    10.0     NaN
     78.1236

In [65]:
dJoin=dHVLTtot.join(dSCOPAAUTtot,on=['ID','TIME'])
dJoin1=dJoin.join(dMOCA,on=['ID','TIME'])
dJoin2=dJoin1.join(dBenton,on=['ID','TIME'])
dJoin3=dJoin2.join(dADL,on=['ID','TIME'])

In [ ]:
df = df.set_index(['ID', 'TIME'])
indices = [idx for idx in df.index.unique('ID') if df.loc[idx].shape[0] >= 2]
df = df[df.index.get_level_values(0).isin(indices)]

In [38]:
print(dJoin3.head(4))

                HVLT_TOTAL  SCOPA_TOTAL  MOCA_TOTAL  BENTON_JLO  \
ID   TIME                                                         
3000 70.123631    0.866667     0.082192         NaN    1.000000   
     71.123631    0.783333     0.123288    0.966667    1.000000   
     72.123631    0.833333     0.123288    0.933333    0.933333   
     73.123631    0.850000     0.109589    1.000000    0.866667   

                SCHWAB_AND_ENGLAND  
ID   TIME                           
3000 70.123631                 NaN  
     71.123631                 NaN  
     72.123631                 NaN  
     73.123631                 NaN  


In [45]:
print(dJoin3.shape)

(6915, 5)


In [66]:
dJoinDropNan=dJoin3.dropna()

In [44]:
print(dJoinDropNan.shape)

(3301, 5)


In [67]:
dJoinDropNan.to_csv("Join5score.csv")

In [36]:
os.chdir("/Users/samuel.gruffaz/Documents/data")


dADNI=pd.read_csv("ADNI_norm.csv", sep=";")

In [37]:
dADNI.rename(columns={"RID":"ID", "AGE_vis":"TIME"}, inplace=True)
dADNI.set_index(["ID", "TIME"], inplace=True)
print(dADNI.shape)

(13226, 364)


In [41]:
print(dADNI.head(5))

            Phase VISCODE2    EXAMDATE  YEARS_SINCE_bl  DX_seq  DX_ffill  \
ID TIME                                                                    
2  74.300   ADNI1       bl  2005-09-08        0.000000     101       0.0   
   74.790   ADNI1      m06  2006-03-06        0.490075     101       0.0   
   77.268   ADNI1      m36  2008-08-27        2.967830     101       0.0   
   79.338  ADNIGO      m60  2010-09-22        5.037650     101       0.0   
   80.329   ADNI2      m72  2011-09-19        6.028750     101       0.0   

           MES_PET_AV1451_INFCEREBGM_REF_SUVR  MES_PET_AV1451_L_ENT_SUVR  \
ID TIME                                                                    
2  74.300                                 NaN                        NaN   
   74.790                                 NaN                        NaN   
   77.268                                 NaN                        NaN   
   79.338                                 NaN                        NaN   
   80.329  

In [16]:
print(dADNI.shape)

(13226, 1)


In [30]:
print(dADNI.loc[3]["DX_seq"])

KeyError: 'DX_seq'

In [35]:
alz2_index=dtadpole["DX_seq"].astype(str).str.contains("2")
#alz2_index = dtadpole["DX_seq"]==2
print(dtadpole.shape)
Tadodo=dtadpole[alz2_index]
Tadodo=Tadodo[["ADAS_TOT13","MMSE_TOT", "MES_MRI_HIPPOCAMPUS_ICV", "MES_MRI_VENTRICLES_ICV","MES_MRI_ENTORHINAL_ICV","MES_CSF_PTAU"]]
print(Tadodo.shape)
#print(set(list(Tadodo["Phase"].values)))
print(len(Tadodo.groupby('ID')))
os.chdir("/Users/samuel.gruffaz/Documents/data")
#Tadodo.to_csv("TAD_AILZ_ADAS_MMSE_HIPP_VENT_ENTHO_PTAU.csv")


(9808, 61)
(3715, 6)
666


In [42]:
alz_index = dADNI["DX_seq"].astype(str).str.contains("2")
#alz_index = dADNI["DX_seq"]==2

AILZ=dADNI[alz_index]
print(set(list(AILZ["Phase"].values)))
AILZ1=AILZ[["MMSE_TOT", "ADAS_TOT13", "MES_MRI_HIPPOCAMPUS_ICV", "MES_MRI_VENTRICLES_ICV",
             "MES_MRI_ENTORHINAL_ICV", "MES_CSF_PTAU", "MES_CSF_ABETA42"
            ] ]
print(len(AILZ1.groupby('ID')))
print(AILZ1.shape)


{'ADNI1', 'ADNIGO', 'ADNI2', 'ADNI3'}
783
(4689, 7)


In [63]:
AILZ.columns

Index(['Phase', 'VISCODE2', 'EXAMDATE', 'YEARS_SINCE_bl', 'DX_seq', 'DX_ffill',
       'MES_PET_AV1451_INFCEREBGM_REF_SUVR', 'MES_PET_AV1451_L_ENT_SUVR',
       'MES_PET_AV1451_L_ENT_VOL', 'MES_PET_AV1451_R_ENT_SUVR',
       ...
       'MES_PET_FDG_AVG_TEMPORAL', 'MES_PET_FDG_AVG', 'MES_CSF_TTAU',
       'MES_CSF_PTAU', 'MES_CSF_PTAU_TTAU', 'MES_CSF_ABETA42',
       'MES_CSF_TTAU_ABETA42', 'MES_CSF_PTAU_ABETA42', 'MES_CSF_ABETA40',
       'MES_CSF_ABETA42_40'],
      dtype='object', length=364)

In [58]:
AILZ2.index.unique('ID')

Int64Index([   3,   10,   29,   76,   83,   84,   91,   93,   94,  129,
            ...
            6264, 6303, 6389, 6545, 6549, 6573, 6601, 6602, 6648, 6655],
           dtype='int64', name='ID', length=236)

In [6]:
AILZ2.to_csv("AILZ_no_filtre.csv")

In [62]:
AILZ1=AILZ[["MMSE_TOT", "ADAS_TOT13", "AVLT_A1t5_SUM_OK", "MES_MRI_HIPPOCAMPUS_ICV", "MES_MRI_VENTRICLES_ICV",
             "MES_MRI_ENTORHINAL_ICV", 
            ] ]
print(AILZ1.shape)
AILZ2=AILZ1.dropna()
print(AILZ2.shape)
AILZ2.to_csv("AILZ_filtre_sansbeta.csv")

(1438, 6)
(981, 6)


In [65]:
AILZ1=AILZ[["MMSE_TOT", "MES_MRI_HIPPOCAMPUS_ICV", "MES_MRI_VENTRICLES_ICV",
             "MES_MRI_ENTORHINAL_ICV", "MES_MRI_AMYGDALA_ICV"
            ] ]
print(AILZ1.shape)
AILZ2=AILZ1.dropna()
print(AILZ2.shape)
AILZ2.to_csv("AILZ_amy_sansbeta.csv")

(1438, 5)
(1071, 5)


In [19]:
AILZ1=AILZ[["MES_MRI_HIPPOCAMPUS_ICV", "MES_MRI_VENTRICLES_ICV",
             "MES_MRI_ENTORHINAL_ICV", "MES_MRI_AMYGDALA_ICV"
            ] ]
print(AILZ1.shape)
AILZ2=AILZ1.dropna()
print(len(AILZ2.groupby('ID')))

AILZ2.to_csv("AILZ_atro.csv")

(1438, 4)
383


In [43]:
AILZ1=AILZ[["MMSE_TOT", "MES_MRI_HIPPOCAMPUS_ICV", "MES_MRI_VENTRICLES_ICV","ADAS_TOT13" 
            ] ]
print(AILZ1.shape)
#AILZ2=AILZ1.dropna()
#print(len(AILZ2.groupby('ID')))
#print(AILZ2.shape)
AILZ1.to_csv("AILZ_MMSE_HIPP_VEN_ADAS.csv")

(4689, 4)


In [28]:
#print(AILZ2.index)
z=0
compt=0
L=[]
for e in AILZ2.index:
    a,b=e
    if a!=z:
        if compt!=0:
            L.append(compt)
        z=a
        compt=1
    compt=compt+1

A=np.array(L)

print(np.mean(A))
print(np.std(A))
print(np.max(A))
print(np.min(A))
    



3.8010471204188483
1.2288840350503603
10
2
